In [1]:
import math
import time
import numpy as np
import json
import glob
import warnings
import requests

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import geopandas as gpd
pd.set_option('display.max_columns', None)


import datetime



C:\Users\enriq\AppData\Roaming\Python\Python39\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\enriq\AppData\Local\Temp\ipykernel_3360\727625543.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.r

# Import der Daten

In [2]:
df1 = pd.read_excel("data/DatenSisag/Verkehrsdaten/Rohdaten Auswertungfiles xls/2022/6009_2022.xlsx")

# Datumszeile erstellen

In [3]:
df1.HHMM = df1.HHMM / 100
df1['HHMM']=df1['HHMM'].replace(24.0,0.0)
df1.head(100)
df1['Datum und Zeit'] = df1["Datum"].astype(str)+ ' ' +df1.HHMM.astype(str)
df1['Datumszeit'] = pd.to_datetime(df1['Datum und Zeit'], format='%Y-%m-%d %H.%M')

In [4]:
df1

,Zst,Datum,HHMM,Richtung,-,Klasse,CS 1,CS 2,CS 3,CS 4,CS 5,CS 6,CS 7,CS 8,CS 9,CS10,Day Helper,Month Helper,Querschnitt,Wochenende/Feirtag,Datum und Zeit,Datumszeit
0,6009,2022-01-01,1.0,1,0,1,0,18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,7,1,NaN,x,2022-01-01 1.0,2022-01-01 01:00:00
1,6009,2022-01-01,1.0,1,0,2,0,0,18,0,0.0,0.0,0.0,0.0,0.0,0.0,7,1,18.0,x,2022-01-01 1.0,2022-01-01 01:00:00
2,6009,2022-01-01,1.0,2,0,1,0,21,0,0,NaN,NaN,NaN,NaN,NaN,NaN,7,1,NaN,x,2022-01-01 1.0,2022-01-01 01:00:00
3,6009,2022-01-01,1.0,2,0,2,0,0,21,0,0.0,0.0,0.0,0.0,0.0,0.0,7,1,21.0,x,2022-01-01 1.0,2022-01-01 01:00:00
4,6009,2022-01-01,2.0,1,0,1,1,38,0,0,NaN,NaN,NaN,NaN,NaN,NaN,7,1,NaN,x,2022-01-01 2.0,2022-01-01 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35031,6009,2022-12-31,23.0,2,0,2,0,1,26,0,0.0,0.0,0.0,0.0,0.0,0.0,7,12,27.0,x,2022-12-31 23.0,2022-12-31 23:00:00
35032,6009,2022-12-31,0.0,1,0,1,1,22,0,0,NaN,NaN,NaN,NaN,NaN,NaN,7,12,NaN,x,2022-12-31 0.0,2022-12-31 00:00:00
35033,6009,2022-12-31,0.0,1,0,2,0,1,22,0,0.0,0.0,0.0,0.0,0.0,0.0,7,12,23.0,x,2022-12-31 0.0,2022-12-31 00:00:00
35034,6009,2022-12-31,0.0,2,0,1,0,14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,7,12,NaN,x,2022-12-31 0.0,2022-12-31 00:00:00
